In [2]:
import pandas as pd
import numpy as np
import time
import csv
import os

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
#load in input (volume or HE) and sex data
#input data contains volume or HE for all subjects x ROIs based on the atlas being used
data = pd.read_csv('input_data.csv', header=0)
data = data.values
#sex is binary variable - males are '1', females are '0'
sex = pd.read_csv('sex.txt', header=None)
sex = sex.values
#read in subj_num  indices
num = pd.read_csv('num.txt', header=None)
num = num.values

In [ ]:
#create logical variable to identify males and females
male = sex==1
female = sex==0

#save male and female he data into separate arrays
#idx_male = num[male.ravel(),:]
#idx_female = num[female.ravel(),:]

In [4]:
#load indices for all test holdout sets - to keep it consistent across all atlases
test_indices = pd.read_csv('test_indices.txt', header=None)
test_indices = test_indices.values

In [ ]:
#set the number of random permutations you want to do, should be the same value as 'iterations' in he_sexclf_traintestsplit
permutations = perm

#size of test_set for each permutation
test_num=test_indices.shape[1]

#create variable to save optimised_c from each of the iterations 
optimised_c = np.zeros(permutations)
accuracy = np.zeros(permutations)
auc = np.zeros(permutations)
predictions = np.zeros([permutations,test_num])
feat_imp = np.zeros([permutations,data.shape[1]])


for idx in range(permutations):
    
    print("Permutation %d" % (idx + 1))
    print(time.localtime(time.time()))
    
    #randomly generate subset of n males and n females
    #n=num_random
    #idx_male_n = idx_male[np.random.choice(idx_male.shape[0], n, replace=False), :]
    #idx_female_n = idx_female[np.random.choice(idx_female.shape[0], n, replace=False), :]
    
    #create logical vector to identify males and females chosen by random permutation
    #male_idx = np.isin(num,idx_male_n)
    #female_idx = np.isin(num,idx_female_n)
    
    #generate arrays of HE or volume data for males and females
    #data_male_n = data[male_idx.ravel(),:]
    #data_female_n = data[female_idx.ravel(),:]
    
    #generate arrays of sex labels for males and females
    #sex_male = sex[male_idx.ravel(),:]
    #sex_female = sex[female_idx.ravel(),:]
    
    #concatenate male and female data together
    #idx_2n = np.concatenate([idx_male_n,idx_female_n])
    #data_2n = np.concatenate([data_male_n,data_female_n])
    #sex_2n = np.concatenate([sex_male,sex_female])
    
    #save subject indices
    #subj_idx[idx,:] = idx_422.ravel()
    
    #split data into train and test subsets
    #x_train, x_test, y_train, y_test = train_test_split(data, sex, test_size = 0.2, stratify = sex, shuffle=True)
    
    #split data into training and testing sets using pre-determined indices (generated in HCP_HE_sexclf_traintestsplit notebook)
    #this was done this way to make sure all atlases have the exact same train-test split 
    
    test=test_indices[idx,:]

    #get indices for training and testing subsets
    train_idx=np.isin(num,test,invert=True)
    test_idx=np.isin(num,test)

    #partition data and sex labels into training and testing subsets
    x_train=data[train_idx.ravel(),:]
    x_test=data[test_idx.ravel(),:]
    y_train=sex[train_idx.ravel(),:]
    y_test=sex[test_idx.ravel(),:]
    
    
    #set all the hyperparameters you want to tune using nested CV
    #course search
    #param_grid = {'clf__kernel': ['linear'], 'clf__C': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]}
    #fine search
    param_grid = {'clf__kernel': ['linear'], 'clf__C': [x*0.001 + 0.001 for x in range(500)]}

    #set number of iterations through nested CV loop
    iterations=100
    
    #set up pipeline for analysis so that data transformation takes place within each CV fold
    cv_steps = [('scaler', StandardScaler()), ('clf', SVC(max_iter=100000))]
    cv_pipeline = Pipeline(cv_steps)

    #create array to store nested CV scores
    nested_scores = np.zeros(iterations)

    best_params = ['']*iterations

    #set up nested CV pipeline
    for iter in range(iterations):
    
        print("Nested CV - Loop %d" % (iter + 1))

        #set parameters for inner and outer loops for CV
        #using 5 fold but can do more/less folded if desired
        inner_cv = StratifiedKFold(n_splits = 5, shuffle = True)
        outer_cv = StratifiedKFold(n_splits = 5, shuffle = True)
    
    
        #print specific hyperparameter being tuned
        #print("Tuning hyperparameters for %s" % score)

        #define classifier with pipeline and grid-search CV for inner loop
        clf = GridSearchCV(cv_pipeline, param_grid = param_grid, cv = inner_cv, 
                           scoring='roc_auc', n_jobs = -1, iid = False, refit = True, verbose=0)

        #fit classifier
        clf.fit(x_train, y_train.ravel())

        #save parameters corresponding to the best score
        best_params[iter] = clf.best_params_
        #print("Best Parameters = %s" % clf.best_params_)
        #print()

        #print detailed classification report
        #print("Detailed classification report:")
        #print("The model is trained on the full development set.")
        #print("The scores are computed on the full validation set.")
        #print()
        #y_true, y_pred = y_val, clf.predict(x_val)
        #print(classification_report(y_true, y_pred))

        #call cross_val_score for outer loop
        nested_score = cross_val_score(clf, X = x_train, y = y_train.ravel(), cv = outer_cv, 
                                       scoring = 'roc_auc', verbose = 0)
        nested_scores[iter] = nested_score.mean()

   #save best_params in txt file
    with open('best_params.txt', 'w') as f:
        for listitem in best_params:
            f.write('%s\n' % listitem)
        
    #extract best C parameter from txt file saved above from all iterations and compute mean 
    cv_params = pd.read_csv('best_params.txt', sep='\s+', header=None)
    cv_params = cv_params[1].str.replace(',','')
    best_c = np.zeros(iterations)
    for i in range(len(best_c)):
        best_c[i] = eval(cv_params[i])
    optimised_c[idx] = np.mean(best_c)
    #print("Optimised C = %1.4f" %(optimised[idx]))
    
    #normalize all training data and fit trasnformation to test data
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train_t = scaler.transform(x_train)
    x_test_t = scaler.transform(x_test)
    
    #generate SVC model based on optimised parameters 
    #uses optimised_c hyperparameter optained by averaging hyperparameters over grid search iterations
    model = SVC(C = optimised_c[idx], kernel='linear', probability=True);
    model.fit(x_train_t, y_train.ravel());
    
    #generate prediction probailities for test samples
    pred = model.predict_proba(x_test_t);
    predictions[idx,:] = pred[:,1]
    
    #save y_test so ROC curves can be generated later
    #test_y[idx,:] = y_test.ravel()
    
    #determine model accuracy and store it in score variable
    accuracy[idx] = model.score(x_test_t,y_test.ravel())
    #print("Accuracy = %1.4f" %(accuracy[idx]))
    
    #determine model AUC and store it in AUC variable
    auc[idx] = metrics.roc_auc_score(y_test.ravel(), pred[:,1]) 
    #print("AUC = %1.4f" %(auc[idx]))
    
    #save feature importances
    feat_imp[idx] = model.coef_[0]
    
    #save at the end of each permutation so if something does go wrong, it's all saved 
    #just gets overwritten after each permutation
    np.savetxt('optc.txt', optimised_c, delimiter=',')
    np.savetxt('acc.txt', accuracy, delimiter=',')
    np.savetxt('pred.txt', predictions, delimiter=',')
    np.savetxt('auc.txt', auc, delimiter=',')
    np.savetxt('featimp.txt', feat_imp, delimiter=',')

    
np.savetxt('optc.txt', optimised_c, delimiter=',')
np.savetxt('acc.txt', accuracy, delimiter=',')
np.savetxt('pred.txt', predictions, delimiter=',')
np.savetxt('auc.txt', auc, delimiter=',')
np.savetxt('featimp.txt', feat_imp, delimiter=',')